In [1]:
!rm -r ./data/
!mkdir data

In [1]:
import time

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

Create an example dataframe
===

In [2]:
days = 365*10
n_securities = 3000
df_wide = pd.DataFrame(data=np.random.rand(days, n_securities), index=pd.date_range('2000', periods=days))
df_wide.columns = ['security_{}'.format(i) for i in range(1, n_securities+1)]
df_wide.head()

,security_1,security_2,security_3,security_4,security_5,security_6,security_7,security_8,security_9,security_10,...,security_2991,security_2992,security_2993,security_2994,security_2995,security_2996,security_2997,security_2998,security_2999,security_3000
2000-01-01,0.354541,0.708911,0.499456,0.510337,0.922883,0.033395,0.357226,0.190096,0.860784,0.005599,...,0.291205,0.034257,0.631848,0.131862,0.824423,0.066367,0.744519,0.048967,0.992172,0.814976
2000-01-02,0.279024,0.079413,0.068638,0.853236,0.338223,0.703762,0.227241,0.311834,0.969553,0.904654,...,0.197242,0.583723,0.795671,0.483001,0.784262,0.720391,0.767673,0.172491,0.412795,0.546260
2000-01-03,0.034746,0.953173,0.759384,0.598813,0.098014,0.141438,0.662880,0.755807,0.183857,0.432211,...,0.500687,0.563940,0.815323,0.975221,0.978903,0.543722,0.283110,0.146224,0.012950,0.829734
2000-01-04,0.512492,0.469804,0.754630,0.039456,0.724362,0.644115,0.766490,0.894468,0.513650,0.635828,...,0.430476,0.845222,0.487150,0.183239,0.908685,0.393748,0.090328,0.365420,0.850565,0.429109
2000-01-05,0.980226,0.133487,0.965429,0.275501,0.416805,0.818239,0.111340,0.152094,0.987746,0.818604,...,0.550709,0.552152,0.834462,0.049471,0.153489,0.387220,0.448912,0.580141,0.422198,0.482580


Using parquet to read/write data
===

In [3]:
%time pq.write_table(pa.Table.from_pandas(df_wide), 'data/wide.parquet')

CPU times: user 1.16 s, sys: 154 ms, total: 1.31 s
Wall time: 1.34 s


In [4]:
%time rb_wide = pq.read_table('data/wide.parquet').to_pandas()

CPU times: user 205 ms, sys: 146 ms, total: 351 ms
Wall time: 276 ms


In [5]:
!du -h data/wide.parquet

100M	data/wide.parquet


In [6]:
# reshape wide to tall
%time df_tall = df_wide.stack().reset_index().rename(columns={'level_0': 'date', 'level_1': 'security_id', 0: 'vals'})

CPU times: user 683 ms, sys: 482 ms, total: 1.16 s
Wall time: 1.16 s


In [7]:
%time pq.write_table(pa.Table.from_pandas(df_tall), 'data/tall.parquet')

CPU times: user 2.52 s, sys: 486 ms, total: 3 s
Wall time: 3.04 s


In [8]:
%time rb_tall = pq.read_table('data/tall.parquet').to_pandas()

CPU times: user 1.48 s, sys: 809 ms, total: 2.29 s
Wall time: 2.17 s


In [9]:
!du -h data/tall.parquet

129M	data/tall.parquet
